In [1]:
import geemap
import ee
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd

import numpy as np
from glob import glob
import urllib.request
from matplotlib.pyplot import imshow
from PIL import Image,ImageDraw,ImageFont

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    
    

In [2]:
def add_wiw(img):
    wiw = img.expression(
        '((B8A/10000) <= 0.1804) && ((B12/10000) <= 0.1131)',{'B8A':img.select('B8A'),'B12':img.select('B12')}
    ).rename('RWS')
    return img.addBands(wiw)

def add_rws(img):
    mndwi = img.normalizedDifference(['B3','B11']).rename('MNDWI')
    mgrn = img.select(['B3','B4','B8']).divide(10000).reduce(ee.Reducer.min()).rename('MGRN')
    rws = img.expression(
        '(MNDWI > 0.3) && (MGRN < 0.15)',{'MNDWI':mndwi,'MGRN':mgrn}
    ).rename('RWS')
    return img.addBands(rws)

def apply_rws(img):
    mask = img.select('RWS').eq(1)
    masked_img = img.updateMask(mask)
    return masked_img

def get_labels(img):
    cluster =  img.select('cluster')
    cluster_vector = cluster.reduceToVectors(reducer = ee.Reducer.countEvery(),geometry=img.geometry(),bestEffort=True,scale=90)
    labels = ee.List(cluster_vector.aggregate_array('label')).distinct().sort()
    return labels

def add_kmeans(img):
    rws_mask = img.select('RWS').eq(1)
    rgb_img = img.select(['B4', 'B3', 'B2']).updateMask(rws_mask)
    
    train_samples = rgb_img.sample(scale=10,numPixels=2000,seed=42,tileScale=4,dropNulls=True)
    k_means = ee.Clusterer.wekaKMeans(8).train(train_samples)
    img_cluster = rgb_img.cluster(k_means).rename('cluster')
    return img.addBands(img_cluster)

def plot_mask(data,layer_name):
    img = data.select(layer_name)
    img_mask = img.updateMask(img.eq(1))
    Map.addLayer(img_mask, {'palette': 'red'}, f'{layer_name}')
    
def plot_rgb(img,text,bands = ['B4', 'B3', 'B2']):
    viz_rgb = {'bands': bands,'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' RGB')
    
def plot_false(img,text):
    viz_rgb = {'bands': ['B6', 'B5', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' False')
    
def get_dates(data):
    date= ee.Image(data).get('GENERATION_TIME')
    return date

def add_mnws(img):

    band_names = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']
    multiband_img = img.select(band_names)
    cluster_img = img.select('cluster')

    samples = multiband_img.addBands(cluster_img).stratifiedSample(numPoints=100,classBand='cluster',scale=10,tileScale=4,seed=42,dropNulls=True)

    nws_list = []
    for label in list(range(0,8)):

        scores = []

        for band in band_names:
            img_band_mu = samples.filter(ee.Filter.eq('cluster',label)).aggregate_mean(band)
            img_band_std = samples.filter(ee.Filter.eq('cluster',label)).aggregate_total_sd(band)

            img_band_raw = multiband_img.select(band)
            img_band_score = img_band_raw.subtract(img_band_mu).divide(img_band_std).pow(2).rename(f'{label} {band}')
            
            scores.append(img_band_score)

        nws = ee.Image.cat(scores).reduce('sum').divide(len(band_names)).sqrt().rename('NWS')
        nws_list.append(nws)

    mnws = ee.Image.cat(nws_list).reduce('min').rename('MNWS')
    return mnws

def get_wd(img):
    img_mnws = img.select('MNWS')
    wd_i = img.expression('MNWS < 3',{'MNWS':img_mnws})
    return wd_i

def count_water_pixel(img):
    count = img.select('RWS').gt(0).reduceRegion(ee.Reducer.sum(),scale=10,maxPixels=1e5,bestEffort=True,tileScale=4).values().get(0)
    return img.set({'water_pixel': count})

def export_frames(img_col,dir_name,annot_list):
    
    print(f'Exporting {len(annot_list)} frames...')
    args = {'dimensions': 480,'bands': ['B11', 'B8', 'B3'],'gain': [0.08, 0.08, 0.08],'format':'png'}
    url = img_col.getFilmstripThumbURL(args)
    
    os.makedirs(dir_name,exist_ok=True)

    saved_strips = urllib.request.urlretrieve(url,f'{dir_name}.png')[0]
    
    img_arr = np.array(Image.open(saved_strips))
    img_stack = img_arr.reshape(len(annot_list),img_arr.shape[0]//len(annot_list),img_arr.shape[1],img_arr.shape[2])
    
    for i in range(len(annot_list)):
        img_pil = Image.fromarray(img_stack[i])
        ImageDraw.Draw(img_pil).text((0, 0),annot_list[i],font = ImageFont.truetype("arial.ttf", 20),fill=(0,0,0))
        fn = f"{dir_name}/{dir_name.split('/')[-1]}_{i}.png"  
        img_pil.save(fn)
        
    os.remove(saved_strips)
    print(f'{len(annot_list)} frames exported in {dir_name}')

In [3]:
#set up GEE map canvas (will be updated with every addLayer() method)
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)
Map.add_basemap('SATELLITE')
# Map.add_basemap('CartoDB.DarkMatter')

#study area
file_path = './data/boundaries/ramsar_stlucia_bbox_clip.geojson'
with open(file_path) as f:
    coord = json.load(f)['features'][0]['geometry']['coordinates'][0]
    line_geom = ee.Geometry.LineString(coord)
    
    #polygon used for clipping purposes
    poly_geom = ee.Geometry.Polygon(coord)
    
Map.addLayer(ee_object=line_geom, vis_params={'color':'red'}, name="Lesser Isimangaliso Wetland Park")
Map


Map(center=[-27.93186, 32.48897], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [ ]:
jrc_gsw_s = ee.ImageCollection("JRC/GSW1_1/YearlyHistory").filter(ee.Filter.eq('year',2017)).first().clip(poly_geom)
Map.addLayer(jrc_gsw_s,{'min': 0, 'max': 3,'palette':['cccccc','ffffff','99d9ea','0000ff']},'JRC GSWS-S 2017')

# jrc_gsw_s_m = ee.ImageCollection("JRC/GSW1_1/MonthlyHistory").filter(ee.Filter.eq('year',2017)).filter(ee.Filter.eq('month',1)).first().clip(poly_geom)
# Map.addLayer(jrc_gsw_s_m,{'min': 0, 'max': 2,'palette':['cccccc','ffffff','0000ff']},'JRC GSWS-S 2017 January')

In [4]:
#get Sentinel-2 Level-1C collection for 2017

cl_pct = 60

s2_col = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(poly_geom).filterDate('2017-01-01','2017-12-31') \
    .filter(ee.Filter.inList('MGRS_TILE', ['36JVP','36JVQ'])) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cl_pct))

# #filter duplicated products 
s2_col_list = s2_col.toList(s2_col.size())

print('Nr. images found: ', s2_col.size().getInfo())


Nr. images found:  147


In [5]:
unique_dates = s2_col_list.map(get_dates).distinct()

#iterave over collection and create mosaic based on similar datetime
col = []
for i in range(unique_dates.size().getInfo()):
    unique_date = unique_dates.get(i)
    date_day = ee.Date(unique_dates.get(i)).format('yyyy-MM-dd')
    date_month = ee.Date(unique_dates.get(i)).format('yyyy-MM')
    similar_tiles = s2_col.filter(ee.Filter.eq('GENERATION_TIME', unique_date)).mosaic().clip(poly_geom).set({'DateDay':date_day,'Month':date_month})
    col.append(similar_tiles)
    
s2_mosaic_col_a = ee.ImageCollection(col)

#iterate over mosaic again and merge based on similar days
date_days = ee.List(s2_mosaic_col_a.aggregate_array('DateDay')).getInfo()
days,count = np.unique(date_days, return_counts=True)
dups = days[count>1]
non_dups = np.setdiff1d(days,dups)

dup_mos = ee.ImageCollection([s2_mosaic_col_a.filter(ee.Filter.eq('DateDay',dup)).mosaic().clip(poly_geom).set({'DateDay':dup}) for dup in dups.tolist()])
non_dup_mos = s2_mosaic_col_a.filter(ee.Filter.inList('DateDay', non_dups.tolist()))

s2_mosaic_col_clean = non_dup_mos.merge(dup_mos).sort('DateDay')

print('Nr. mosaics created: ', len(date_days),len(days))

Nr. mosaics created:  80 79


In [ ]:
# #export mosaics as png
# dir_name = './data/ts_animation/2019_cl60'
# export_frames(s2_mosaic_col_clean,dir_name,days.tolist())

In [6]:
#select best mosaics 
cleaned_dir_name = './data/ts_animation/2017_cl60_cleaned_v1'
selected_indices = np.sort(np.array(list(map(lambda x:x.split('_')[-1].split('.')[0],os.listdir(cleaned_dir_name)))).astype(int))
selected_days = days[selected_indices].tolist()
print(len(selected_days),'mosaics selected')

#filter best mosaic from collection
s2_mosaic_best = s2_mosaic_col_clean.filter(ee.Filter.inList('DateDay', selected_days))
s2_mosaic_best_list = s2_mosaic_best.toList(s2_mosaic_best.size())

51 mosaics selected


In [7]:
first = s2_mosaic_best.first()

In [9]:
first.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1, 2],
   'origin': [32, -29],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1, 2],
   'origin': [32, -29],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1, 2],
   'origin': [32, -29],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1, 2],
   'origin': [32, -29],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    

In [ ]:
pd.DataFrame(pd.Series(selected_days),columns=['2019']).to_csv('./data/ts_animation/2019_cl60_selected.csv')

In [ ]:
mnws_rws_25feb = add_mnws(add_kmeans(add_rws(s2_mosaic_best.first())))
mnws_wiw_25feb = add_mnws(add_kmeans(add_wiw(s2_mosaic_best.first())))
file_path = './data/boundaries/landcover_25feb.geojson'
with open(file_path) as f:
    coords = json.load(f)
    features = geojson_to_ee(coords)
    labels = features.aggregate_array('type').distinct()
    labels_int = ee.List([1,2,3,4,5,6])
    features_remapped = features.remap(labels,labels_int,'type')
    label_class = dict(zip(labels_int.getInfo(),labels.getInfo()))

In [ ]:
mnws_rws_25feb_samples = mnws_rws_25feb.sampleRegions(collection= features_remapped,properties= ['type'],scale= 10,tileScale=4)
mnws_wiw_25feb_samples = mnws_wiw_25feb.sampleRegions(collection= features_remapped,properties= ['type'],scale= 10,tileScale=4)

In [ ]:
mnws_values = mnws_rws_25feb_samples.aggregate_array('MNWS').getInfo()
labels_values = mnws_rws_25feb_samples.aggregate_array('type').getInfo()

data = pd.DataFrame(np.array([mnws_values,labels_values])).T
data.columns = ['MNWS','label']
data['label'] = data['label'].astype(int)
data['label'] = data['label'].map(label_class)
data.boxplot(column='MNWS',by='label',grid=False,rot=45, fontsize=20,figsize=(10,5))

for lab in data.label.unique():
    print(data[data['label']==lab].describe().loc[['min','mean','max','std']].T,lab)

In [ ]:
mnws_values = mnws_wiw_25feb_samples.aggregate_array('MNWS').getInfo()
labels_values = mnws_wiw_25feb_samples.aggregate_array('type').getInfo()

data = pd.DataFrame(np.array([mnws_values,labels_values])).T
data.columns = ['MNWS','label']
data['label'] = data['label'].astype(int)
data['label'] = data['label'].map(label_class)
data.boxplot(column='MNWS',by='label',grid=False,rot=45, fontsize=20,figsize=(10,5))

for lab in data.label.unique():
    print(data[data['label']==lab].describe().loc[['min','mean','max','std']].T,lab)

In [ ]:
#map MNWS timeseries with RWS (original)
mnws_ts_rws = s2_mosaic_best.map(add_rws).map(add_kmeans).map(add_mnws)

#map MNWS timeseries with WIW 
mnws_ts_wiw = s2_mosaic_best.map(add_wiw).map(add_kmeans).map(add_mnws)

In [ ]:
#calculate IWF with RWS
img_wf_rws = mnws_ts_rws.map(get_wd).reduce(reducer=ee.Reducer.sum(),parallelScale=8).divide(len(selected_days))

#calculate IWF with WIW
img_wf_wiw = mnws_ts_wiw.map(get_wd).reduce(reducer=ee.Reducer.sum(),parallelScale=8).divide(len(selected_days))

In [ ]:
# export all bands stack to one geotiff
geemap.ee_export_image(img_wf_rws, filename='./data/iwf/iwf_rws_39.tif', scale=90, region=poly_geom, file_per_band=False)
geemap.ee_export_image(img_wf_wiw, filename='./data/iwf/iwf_wiw_39.tif', scale=90, region=poly_geom, file_per_band=False)

remove_zips = list(map(lambda x:os.remove(x),glob('./data/iwf/*.zip')))

In [ ]:
#export image collection as geotiff 
# geemap.ee_export_image_collection(mnws_ts, out_dir='./data/mnws_ts_wiw',region=poly_geom,scale=90)
# remove_zips = list(map(lambda x:os.remove(x),glob('./data/mnws_ts_wiw/*.zip')))

In [ ]:
# export image to drive
band_names = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']
aoi = ee_to_geojson(poly_geom)['coordinates'][0]

for i in range(len(selected_days)):

    file_name = 'mosaic_'+selected_days[i].replace('-','')
    data = ee.Image(s2_mosaic_best_list.get(i)).select(band_names)
    task = ee.batch.Export.image.toDrive(data, 
                                         description='mosaic',
                                         folder='GEE_output',
                                        fileNamePrefix=file_name,
                                        crs='EPSG:32736',
                                        scale=10,
                                        maxPixels=1e8,
                                        fileFormat='GeoTIFF',
                                        skipEmptyTiles=True,
                                        region=aoi)
    task.start()